# Iceland Snow and Ice Monitoring

This notebook implements a workflow for monitoring snow and ice in Iceland using Sentinel-2 data via the EOPF Zarr format.

In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from shapely.geometry import box
import shapely.geometry
import pystac_client
from pystac import Item
import xarray as xr
import os
import requests
from datetime import datetime
from pyproj import Transformer
import dask
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed

## Seeds
Load the glacier seeds (points) and define the Area of Interest.

In [ ]:
# Load seeds
seeds_gdf = gpd.read_file("data/Iceland_Seeds.geojson")

# Reproject to WGS84 for search
seeds_gdf = seeds_gdf.to_crs("EPSG:4326")

# Get bounding box in WGS84
total_bounds = seeds_gdf.total_bounds
bbox_4326 = list(total_bounds) # [minx, miny, maxx, maxy]

# Define AOI for UTM transformation
spatial_extent = {
    "west": bbox_4326[0],
    "south": bbox_4326[1],
    "east": bbox_4326[2],
    "north": bbox_4326[3],
}

print(f"Bbox (EPSG:4326): {bbox_4326}")

# Convert AOI to UTM 27N (EPSG:32627) - Common for Iceland
# The example used EPSG:32631 for Belgium. For Iceland, we use 32627.
target_crs = "EPSG:32627"
transformer = Transformer.from_crs("EPSG:4326", target_crs, always_xy=True)

west_utm, south_utm = transformer.transform(
    spatial_extent["west"], spatial_extent["south"]
)
east_utm, north_utm = transformer.transform(
    spatial_extent["east"], spatial_extent["north"]
)

# Spatial slice parameters (Note: y is typically north-to-south in these grids, so slice order matters)
# We will verify the order after inspection, but typically it is slice(max_y, min_y) or slice(min_y, max_y) depending on the index.
# The example used slice(north_utm, south_utm) for y, implying descending coordinates.
x_slice = slice(west_utm, east_utm)
y_slice = slice(north_utm, south_utm)

print(f"UTM Bounds ({target_crs}): West={west_utm}, South={south_utm}, East={east_utm}, North={north_utm}")

## STAC Search and Data Loading

This algorithm simulates a file-based workflow by processing full Sentinel-2 scenes (Sentinel-2 L2A) retrieved from the EOPF Zarr store. 
It avoids tile-based optimization and instead loads the full scene extent to compute NDSI and classify snow.

In [ ]:
# === Configuration ===
SINGLE_SCENE_MODE = True  # Toggle: True = only first scene, False = all scenes

# STAC Search
catalog = pystac_client.Client.open("https://stac.core.eopf.eodc.eu")

# Search for Sentinel-2 L2A items
time_range_str = "2025-09-01/2025-09-03"

print(f"Searching STAC for {time_range_str} over AOI...")
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox_4326,
    datetime=time_range_str,
)

items = list(search.items())
print(f"Found {len(items)} items.")

# Filter: only non-deprecated items with 'product' asset
valid_items = [
    item for item in items 
    if not item.properties.get("deprecated", False) and "product" in item.assets
]
print(f"Valid items with product asset: {len(valid_items)}")

# Get hrefs for file-based processing
hrefs = [item.assets["product"].href for item in valid_items]

# Apply single scene toggle
if SINGLE_SCENE_MODE and hrefs:
    hrefs = [hrefs[0]]
    print(f"\n[SINGLE_SCENE_MODE] Using only first scene: {os.path.basename(hrefs[0])}")
elif hrefs:
    print(f"\nTotal scenes to process: {len(hrefs)}")

## File-Based Scene Processing

Load and process each Sentinel-2 scene individually. This simulates a file-based workflow where each scene is processed one at a time.

In [ ]:
def load_scene(href, x_slice, y_slice):
    """
    Load a single Sentinel-2 scene from EOPF Zarr store.
    Returns bands B03 (Green), B11 (SWIR), and SCL mask aligned to 10m grid.
    """
    scene_name = os.path.basename(href.rstrip("/"))
    print(f"Loading scene: {scene_name}")
    
    # Load Green Band (B03) - 10m resolution
    ds_b03 = xr.open_zarr(href, group="/measurements/reflectance/r10m")[["b03"]]
    ds_b03 = ds_b03.sel(x=x_slice, y=y_slice)
    
    # Load SWIR Band (B11) - 20m resolution
    ds_b11 = xr.open_zarr(href, group="/measurements/reflectance/r20m")[["b11"]]
    ds_b11 = ds_b11.sel(x=x_slice, y=y_slice)
    
    # Load SCL (Scene Classification) - 20m resolution
    ds_scl = xr.open_zarr(href, group="/conditions/mask/l2a_classification/r20m")[["scl"]]
    ds_scl = ds_scl.sel(x=x_slice, y=y_slice)
    
    # Resample 20m bands to 10m grid (align to B03)
    ds_b11_interp = ds_b11.interp_like(ds_b03, method="nearest")
    ds_scl_interp = ds_scl.interp_like(ds_b03, method="nearest")
    
    # Merge into single dataset
    scene_data = xr.merge([ds_b03, ds_b11_interp, ds_scl_interp])
    
    # Extract datetime from filename
    parts = scene_name.split("_")
    date_str = next((p for p in parts if p.startswith("20") and "T" in p), None)
    if date_str:
        scene_data.attrs["datetime"] = datetime.strptime(date_str.split(".")[0], "%Y%m%dT%H%M%S")
    
    scene_data.attrs["scene_name"] = scene_name
    return scene_data


def compute_ndsi(scene_data):
    """
    Compute NDSI (Normalized Difference Snow Index) and snow mask.
    NDSI = (Green - SWIR) / (Green + SWIR)
    Snow threshold: NDSI > 0.42
    """
    green = scene_data["b03"]
    swir = scene_data["b11"]
    scl = scene_data["scl"]
    
    # Calculate NDSI
    denom = green + swir
    ndsi = (green - swir) / denom.where(denom != 0)
    
    # Cloud mask: exclude Cloud Shadows (3), Cloud Medium (8), Cloud High (9)
    valid_mask = ~scl.isin([3, 8, 9])
    
    # Snow classification (NDSI > 0.42 and valid pixel)
    snow_mask = (ndsi > 0.42) & valid_mask
    
    return xr.Dataset({
        "ndsi": ndsi,
        "snow_mask": snow_mask,
        "valid_mask": valid_mask
    }, attrs=scene_data.attrs)


def process_scenes(hrefs, x_slice, y_slice):
    """
    Process all scenes in file-based workflow.
    Returns list of results (one per scene).
    """
    results = []
    
    for i, href in enumerate(hrefs):
        print(f"\n[{i+1}/{len(hrefs)}] Processing...")
        try:
            # Load scene
            scene_data = load_scene(href, x_slice, y_slice)
            
            # Compute NDSI and snow mask
            result = compute_ndsi(scene_data)
            
            # Trigger computation
            result = result.compute()
            
            # Statistics
            snow_pixels = result["snow_mask"].sum().item()
            valid_pixels = result["valid_mask"].sum().item()
            snow_percent = (snow_pixels / valid_pixels * 100) if valid_pixels > 0 else 0
            
            print(f"  → Snow pixels: {snow_pixels:,} ({snow_percent:.1f}% of valid area)")
            
            results.append(result)
            
        except Exception as e:
            print(f"  → Error: {e}")
            continue
    
    return results


# Process scenes
if hrefs:
    print(f"Starting file-based processing of {len(hrefs)} scene(s)...\n")
    results = process_scenes(hrefs, x_slice, y_slice)
    print(f"\n✓ Successfully processed {len(results)} scene(s)")
else:
    print("No scenes to process.")